In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time


In [2]:
hop_length = 3
frames = 9
win_length = 4*hop_length
channels = hop_length*2+1
wav_length = hop_length*frames


audio_origin = torch.rand((1,wav_length), dtype=torch.float64)
audio_origin = torch.round(audio_origin*1000).long()
print(audio_origin.shape)
print(audio_origin[0,:10])


torch.Size([1, 27])
tensor([936, 305, 971, 542, 984, 909, 744, 248, 989, 657])


In [3]:
# stft_fn = stft.STFT(filter_length=4, hop_length=1, win_length=4,
#                     window='hann')

stft_fn = stft.STFT(filter_length=win_length, hop_length=hop_length, win_length=win_length,
                    window=None)


def compare(a,b):
    return torch.mean(torch.abs(a-b)), torch.mean((a-b)*(a-b))

def compare_L1(ori,gen):
    return torch.mean(torch.abs(ori-gen)/torch.abs(ori))


def compare_L2(a,b):
    return torch.sum(torch.abs(a-b)), torch.sum((a-b)*(a-b))

def magnitude_to_L(magnitude, forward_basis):
    # magnitude:      [1, 2H+1, T]
    # forward_basis:  [2H+1, 1, 4H]
    
    # output L shape: [2H+1,T]
    H = (magnitude.shape[1]-1)//2
    basis = np.zeros((2*H+1,2*H+1), dtype= np.float64)
    basis[:, :] = forward_basis[:,0,:2*H+1]
    basis_inv = np.linalg.pinv(basis)
    L = np.matmul(basis_inv, magnitude[0,:,:]**2)
    L[1:2*H+1] = L[1:2*H+1]/2
    return L


In [17]:
x = audio_origin[0,:]
S0 = torch.sum(x[0:0+12:3])
S1 = torch.sum(x[1:1+12:3])
S2 = torch.sum(x[2:2+12:3])
S3 = torch.sum(x[3:3+12:3])
S4 = torch.sum(x[4:4+12:3])
S5 = torch.sum(x[5:5+12:3])
S6 = torch.sum(x[6:6+12:3])
S7 = torch.sum(x[7:7+12:3])
S8 = torch.sum(x[8:8+12:3])
S9 = torch.sum(x[9:9+12:3])
S10 = torch.sum(x[10:10+12:3])
S11 = torch.sum(x[11:11+12:3])

C1 = x[13] - x[1]
C02 = x[12]+x[14]-x[0]-x[2]
y0 = x[12]-x[0]

C4 = x[4+12] - x[4]
C35 = x[3+12]+x[5+12]-x[3]-x[5]
y3 = x[3+12]-x[3]
C7 = x[7+12] - x[7]
C68 = x[6+12]+x[8+12]-x[6]-x[8]
y6 = x[6+12]-x[6]

print('C1', C1)
print('C02', C02)
print('C4', C4)
print('C35', C35)
print('C7', C7)
print('C68', C68)


T10 = S0*S1 + S1*S2 + S2*S0
print('T10', T10)
testT10 = (S4-C1)*(S3+S5-C02) + (S3-y0)*(S5-(C02-y0))
print(testT10)
T00 = S0**2 + S1**2 + S2**2
print('T00', T00)
testT00 = (S4-C1)**2 + (S3-y0)**2 + (S5-(C02-y0))**2
print(testT00)

T11 = S3*S4 + S4*S5 + S5*S3
print('T11', T11)
T01 = S3**2 + S4**2 + S5**2
print('T01', T01)

T12 = S6*S7 + S7*S8 + S8*S6
print('T12', T12)
T02 = S6**2 + S7**2 + S8**2
print('T02', T02)

T13 = S9*S10 + S10*S11 + S11*S9
print('T13', T13)
T03 = S9**2 + S10**2 + S11**2
print('T03', T03)


C1 tensor(278)
C02 tensor(-566)
C4 tensor(-339)
C35 tensor(-770)
C7 tensor(547)
C68 tensor(-1128)
T10 tensor(18559646)
tensor(18559646)
T00 tensor(20777589)
tensor(20777589)
T11 tensor(17574996)
T01 tensor(18447049)
T12 tensor(12615781)
T02 tensor(13357382)
T13 tensor(10450188)
T03 tensor(10807785)


In [ ]:
syms y0 y3 C1 C4 C02 C35 x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 L01_L00 L11_L10 L21_L20 L31_L30 L41_L40 L51_L50 L61_L60 L02_L01 L12_L11 L22_L21 L32_L31 L42_L41 L52_L51 L62_L61;

eq00= 0.5*L01_L00==(x0-x2) *y0 + x1*C1      + x2*C02        + 0.5*y0^2 + 0.5*C1^2 + 0.5*(C02-y0)^2;
eq10= L11_L10    ==(x11-x3)*y0 + (x0+x2)*C1 + (x1+x3)*C02   + C1*C02; 
eq20= L21_L20    ==(x10+x2 - x0-x4)*y0 + (x11+x3)*C1 + (x0+x4)*C02   + y0*(C02-y0);
eq30= L31_L30    ==(x9+x3 - x11-x5)*y0 + (x10+x4)*C1 + (x11+x5)*C02;
eq40= L41_L40    ==(x8+x4 - x10-x6)*y0 + (x9+x5)*C1 + (x10+x6)*C02;
eq50= L51_L50    ==(x5-x9)*y0 + (x8+x6)*C1 + (x9+x7)*C02;
eq60= 0.5*L61_L60==(x6-x8)*y0 + x7*C1 + x8*C02;

eq01= 0.5*L02_L01==(x3-x5) *y3 + x4*C4      + x5*C35        + 0.5*y3^2 + 0.5*C4^2 + 0.5*(C35-y3)^2;
eq11= L12_L11    ==(x2-x6)*y3 + (x3+x5)*C4 + (x4+x6)*C35   + C4*C35 + (C02-y0)*y3; 
eq21= L22_L21    ==(x1+x5 - x3-x7)*y3  + (x2+x6)*C4 + (x3+x7)*C35   + y3*(C35-y3) + C1*y3 + (C02-y0)*C4;
eq31= L32_L31    ==(x0+x6 - x2-x8)*y3  + (x1+x7)*C4 + (x2+x8)*C35 + y0*y3 + C1*C4 + (C02-y0)*(C35-y3);
eq41= L42_L41    ==(x11+x7 - x1-x9)*y3 + (x0+x8)*C4 + (x1+x9)*C35 + y0*C4 + C1*(C35-y3);
eq51= L52_L51    ==(x8-x0)*y3 + (x11+x9)*C4 + (x0+x10)*C35 +y0*(C35-y3);
eq61= 0.5*L62_L61==(x9-x11)*y3 + x10*C4 + x11*C35;



[x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11]=solve(eq00,eq10,eq20,eq30,eq40,eq50,eq60, ...,
                                                    eq01,eq11,eq21,eq31,eq41,eq51,eq61, ...,
                                                    x0, x1, x2, x3, x4, x5, ...,
                                                    x6, x7, x8, x9, x10, x11)



In [ ]:
syms y0 y3 y6 C1 C4 C7 C02 C35 C68 s0 s2 s3 s5 F01 F31 F02 F32 F03 F33;

eq01= F01 ==(s0-s2)*y0 + (-s3-s5)*C1 + s2*C02 + y0^2 - C02*y0;
eq31= F31 ==(s3-s5)*y0 + (-s0-s2)*C1 + s5*C02; 
eq02= F02 ==(s3-s5)*y3 + (-s0-s2)*C4 + s5*C35 + y3^2 - C35*y3;
eq32= F32 ==(s0-s2)*y3 + (-s3-s5)*C4 + s2*C35 + (2*y0-C02)*y3 - y0*C35;
eq03= F03 ==(s0-s2)*y6 + (-s3-s5)*C7 + s2*C68 + y6^2 - C68*y6 + (2*y0-C02)*y6 - y0*C68;
eq33= F33 ==(s3-s5)*y6 + (-s0-s2)*C7 + s5*C68 + (2*y3-C35)*y6 - y3*C68;



[s0 s2 s3 s5]=solve(eq01,eq31,eq02,eq32, ...,
                    s0,s2,s3,s5)

s0-s2 =
 
(C02*C4*F03 - C1*C35*F03 - C02*C7*F32 + C7*C35*F01 
 - C4*C68*F01 + C1*C68*F32 - C02*C4*y6^2 + C02^2*C4*y6 
 - C02^2*C7*y3 + C1*C35*y6^2 - C7*C35*y0^2 + C4*C68*y0^2 
 - C1*C02*C35*y6 + C1*C02*C68*y3 + C02*C4*C68*y6 - C1*C35*C68*y6 
 - 2*C02*C4*y0*y6 + 2*C02*C7*y0*y3 + 2*C1*C35*y0*y6 - 2*C1*C68*y0*y3)/
(C02*C4*y6 - C02*C7*y3 - C1*C35*y6 + C7*C35*y0 + C1*C68*y3 - C4*C68*y0)
 
 
-s3-s5 =
 
-(C02*F03*y3 - C35*F03*y0 - C02*F32*y6 + C35*F01*y6 
  - C68*F01*y3 + C68*F32*y0 - C02*y3*y6^2 + C35*y0*y6^2 
  + C35*y0^2*y6 - C68*y0^2*y3 - C02*C35*y0*y6 + C02*C68*y0*y3 
  + C02*C68*y3*y6 - C35*C68*y0*y6)/
(C02*C4*y6 - C02*C7*y3 - C1*C35*y6 + C7*C35*y0 + C1*C68*y3 - C4*C68*y0)
 
 
s2 =
 
(C1*F03*y3 - C4*F03*y0 + C4*F01*y6 - C7*F01*y3 
 - C1*F32*y6 + C7*F32*y0 + C7*C35*y0^2 - C4*C68*y0^2 
 - C1*y3*y6^2 + C4*y0*y6^2 + C4*y0^2*y6 - C7*y0^2*y3 
 - C1*C35*y0*y6 + C1*C68*y0*y3 + C1*C68*y3*y6 - C4*C68*y0*y6)/
(C02*C4*y6 - C02*C7*y3 - C1*C35*y6 + C7*C35*y0 + C1*C68*y3 - C4*C68*y0)


s3-s5 =
 
-(C02*C7*F02 - C02*C4*F33 + C1*C35*F33 - C1*C68*F02 
  - C7*C35*F31 + C4*C68*F31 - C02*C7*y3^2 + C1*C35^2*y6 
  + C1*C68*y3^2 - C02*C4*C35*y6 + C02*C7*C35*y3 - C02*C4*C68*y3 
  + 2*C02*C4*y3*y6 - 2*C1*C35*y3*y6)/
(C02*C4*y6 - C02*C7*y3 - C1*C35*y6 + C7*C35*y0 + C1*C68*y3 - C4*C68*y0)
 
 
-s0-s2 =
 
(C02*F02*y6 - C02*F33*y3 + C35*F33*y0 - C68*F02*y0 
 - C35*F31*y6 + C68*F31*y3 - C02*C68*y3^2 + C02*y3^2*y6 
 + C35^2*y0*y6 + C68*y0*y3^2 - 2*C35*y0*y3*y6)/
(C02*C4*y6 - C02*C7*y3 - C1*C35*y6 + C7*C35*y0 + C1*C68*y3 - C4*C68*y0)
 
 
s5 =
 
-(C1*F02*y6 - C7*F02*y0 - C1*F33*y3 + C4*F33*y0 
  - C4*F31*y6 + C7*F31*y3 - C1*C68*y3^2 + C1*y3^2*y6 
  + C7*y0*y3^2 - 2*C4*y0*y3*y6 + C4*C35*y0*y6 - C7*C35*y0*y3 
  + C4*C68*y0*y3)/
(C02*C4*y6 - C02*C7*y3 - C1*C35*y6 + C7*C35*y0 + C1*C68*y3 - C4*C68*y0)

 

